In [1]:
import requests
import json
import pandas as pd


In [2]:
url ="http://publicplansdata.org/api/"
#http://publicplansdata.org/api/?q=QVariables&variables=fy,PlanName,PercentReqContPaid,InvestmentReturn_5yr,ActFundedRatio_GASB&filterfystart=2000&filterfyend=2019&format=json

In [3]:
params = {
    "q": "ListDataSets",
     "format": "json"
    
}


In [4]:
response = requests.get(url, params = params)
print(response.url)

http://publicplansdata.org/api/?q=ListDataSets&format=json


In [5]:
data = response.json()
data

[{'status': 'OK',
  'date': '2021-05-28 00:48:10',
  'q': 'listdatasets',
  'params': 'q=ListDataSets&format=json',
  'recordcount': 34},
 {'id': '3',
  'table_name': 'pensiongasbschedules',
  'table_display_name': 'Funding'},
 {'id': '4',
  'table_name': 'pensiongasbassumptions',
  'table_display_name': 'Assumptions and Methods'},
 {'id': '5',
  'table_name': 'pensionincomestatement',
  'table_display_name': 'Income Statement'},
 {'id': '7',
  'table_name': 'pensioninvestmentreturn',
  'table_display_name': 'Investment Returns'},
 {'id': '8',
  'table_name': 'pensionactuarialcosts',
  'table_display_name': 'Actuarial Costs'},
 {'id': '9',
  'table_name': 'pensionprovisions',
  'table_display_name': 'Plan Provisions'},
 {'id': '10',
  'table_name': 'pensionmembership',
  'table_display_name': 'Membership'},
 {'id': '11',
  'table_name': 'pensionreportingdates',
  'table_display_name': 'Plan Reporting'},
 {'id': '12',
  'table_name': 'pensioneegroupbasics',
  'table_display_name': 'Empl

In [6]:
#Make sure to use QDataset, NOT QDataSet.  Makes a big difference
paramsratios = {
    "q": "QDataset",
    "dataset": "pensiongasbschedules",
     "format": "json",
    #"variables": 
     #"v": variables,
}
paramsbasicplandata = {
    "q": "QDataset",
    "dataset": "pensionplanbasics",
     "format": "json",
    #"variables": 
     #"v": ,
}


paramsbasicinvestperform = {
    "q": "QDataset",
    "dataset": "pensioninvestmentperformancedetailed_actl",
     "format": "json",
    #"variables": 
     #"v": ,
}

In [7]:
ratiosresponse = requests.get(url, params = paramsratios)
#print(ratiosresponse.url)
plandataresponse = requests.get(url,params=paramsbasicplandata)
investperformresponse = requests.get(url,params=paramsbasicinvestperform)

In [8]:
fundedratiosdata = ratiosresponse.json()
plandata = plandataresponse.json()
investperform = investperformresponse.json()

In [9]:
funded_ratios_full = pd.DataFrame(fundedratiosdata[1:])
plan_data_full = pd.DataFrame(plandata[1:])
investperform_data_full=pd.DataFrame(investperform[1:])

In [10]:
funded_ratios_full.columns

Index(['ActAssets_est', 'ActAssets_GASB', 'ActFundedRatio_est',
       'ActFundedRatio_GASB', 'ActFundedRatio_GASB67', 'ActLiabilities_est',
       'ActLiabilities_GASB', 'ActLiabilities_other', 'ADEC', 'EEGroupID',
       'fy', 'NetPensionLiability', 'NetPosition', 'notes_GASBSchedules',
       'payroll', 'PercentReqContPaid', 'ppd_id', 'RequiredContribution',
       'RequiredContribution_est', 'source_GASBSchedules', 'TierID',
       'TotalPensionLiability', 'UAAL_GASB'],
      dtype='object')

In [11]:
#remove rows with empty funded ratio data
funded_ratio_df=funded_ratios_full[['ppd_id','fy','ActFundedRatio_GASB','ActFundedRatio_est','PercentReqContPaid']].dropna()
funded_ratio_df.shape

(3758, 5)

In [12]:
plan_data_full.columns

Index(['AdministeringGovt', 'BenefitsWebsite', 'CostSharing', 'CostStructure',
       'DataEndYear', 'DataStartYear', 'EmployeeTypeCovered', 'EmployerType',
       'FiscalYearType', 'GovtName', 'InPFS', 'notes_PlanBasics', 'PlanClosed',
       'PlanFullName', 'PlanInceptionYear', 'PlanName', 'PlanType',
       'PlanYearClosed', 'ppd_directory_listed', 'ppd_id', 'SocSecCovered',
       'SocSecCovered_verbatim', 'source_PlanBasics', 'StateAbbrev',
       'StateName', 'system_id'],
      dtype='object')

In [13]:
plan_data_df=plan_data_full[['ppd_id','PlanName','StateAbbrev','PlanClosed','PlanYearClosed',]]
plan_data_df.head()

,ppd_id,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,Alabama ERS,AL,0,None
1,2,Alabama Teachers,AL,0,None
2,3,Alaska PERS,AK,1,2006
3,4,Alaska Teachers,AK,1,2006
4,5,Arizona Public Safety,AZ,0,None


In [14]:
plan_test_data_df=pd.merge(funded_ratio_df,plan_data_df,how="inner",on="ppd_id")
plan_test_data_df['ActFundedRatio_GASB'] = plan_test_data_df['ActFundedRatio_GASB'].astype(float)
plan_test_data_df.shape

(3758, 9)

In [15]:
plan_test_data_df.head(50)

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PercentReqContPaid,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,2001,1.002,0.000,1.00000,Alabama ERS,AL,0,None
1,1,2002,0.954,0.000,1.00000,Alabama ERS,AL,0,None
2,1,2003,0.911,0.000,1.00000,Alabama ERS,AL,0,None
3,1,2004,0.897,0.000,1.00000,Alabama ERS,AL,0,None
4,1,2005,0.840,0.000,1.00000,Alabama ERS,AL,0,None
5,1,2006,0.811,0.000,1.00000,Alabama ERS,AL,0,None
6,1,2007,0.790,0.000,1.00000,Alabama ERS,AL,0,None
7,1,2008,0.757,0.000,1.00000,Alabama ERS,AL,0,None
8,1,2009,0.722,0.000,1.00000,Alabama ERS,AL,0,None
9,1,2010,0.682,0.000,1.00000,Alabama ERS,AL,0,None


In [16]:
plan_data_2020=plan_test_data_df[(plan_test_data_df['fy']=='2020')]
plan_data_2020

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PercentReqContPaid,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
19,1,2020,0.68200,0.000,1.00000,Alabama ERS,AL,0,None
39,2,2020,0.69400,0.000,1.00000,Alabama Teachers,AL,0,None
59,3,2020,0.63700,0.000,1.00045,Alaska PERS,AK,1,2006
79,4,2020,0.75300,0.000,1.16002,Alaska Teachers,AK,1,2006
116,6,2020,0.71700,0.000,1.00000,Arizona SRS,AZ,0,None
...,...,...,...,...,...,...,...,...,...
3623,195,2020,0.92500,0.000,1.00000,Delaware County and Municipal Employees,DE,0,None
3647,197,2020,0.69303,0.000,1.00668,Louisiana Municipal Employees,LA,0,None
3667,199,2020,0.75800,0.000,0.91115,Vermont Municipal Employees,VT,0,None
3687,200,2020,0.77620,0.000,1.01526,St. Louis Employees,MO,0,None


In [17]:
plan_data_2020.sort_values(by='ActFundedRatio_GASB', ascending=True)[0:10]

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PercentReqContPaid,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
3531,189,2020,0.09620,0.000,0.65962,"Charleston, WV Firemen's Pension",WV,1,2011
807,41,2020,0.17200,0.000,0.90984,Kentucky ERS,KY,0,None
3221,169,2020,0.23874,0.000,1.02187,Providence ERS,RI,0,None
318,16,2020,0.38500,0.000,1.00001,Connecticut SERS,CT,0,None
648,33,2020,0.38670,0.000,0.81304,Illinois SERS,IL,0,None
668,34,2020,0.40500,0.000,0.61401,Illinois Teachers,IL,0,None
688,35,2020,0.42230,0.000,0.79981,Illinois Universities,IL,0,None
787,40,2020,0.48241,0.000,0.84548,Kentucky County,KY,0,None
2237,114,2020,0.49170,0.000,1.00590,Vermont Teachers,VT,0,None
2929,152,2020,0.49700,0.000,0.91548,Philadelphia Municipal,PA,0,None


In [18]:
plan_data_20yr=plan_test_data_df[(plan_test_data_df['ActFundedRatio_GASB']>=.9)]
plan_data_20yr

,ppd_id,fy,ActFundedRatio_GASB,ActFundedRatio_est,PercentReqContPaid,PlanName,StateAbbrev,PlanClosed,PlanYearClosed
0,1,2001,1.0020,0.000,1.00000,Alabama ERS,AL,0,None
1,1,2002,0.9540,0.000,1.00000,Alabama ERS,AL,0,None
2,1,2003,0.9110,0.000,1.00000,Alabama ERS,AL,0,None
20,2,2001,1.0140,0.000,1.00000,Alabama Teachers,AL,0,None
21,2,2002,0.9740,0.000,1.00000,Alabama Teachers,AL,0,None
...,...,...,...,...,...,...,...,...,...
3693,201,2006,1.0857,0.000,1.59940,Dallas ERS,TX,0,None
3694,201,2007,1.0920,0.000,2.49420,Dallas ERS,TX,0,None
3695,201,2008,0.9617,0.000,2.10410,Dallas ERS,TX,0,None
3696,201,2009,0.9497,0.000,0.64060,Dallas ERS,TX,0,None


In [19]:
#count number of years that each plan was above 90% funded.
plan_90plus_counts=plan_data_20yr.groupby("PlanName")["fy"].count()
plan_90plus_counts

PlanName
Alabama ERS                      3
Alabama Teachers                 3
Alameda County ERS               2
Alaska PERS                      1
Alaska Teachers                  1
                                ..
Washington Teachers Plan 2/3    19
West Virginia PERS               5
Wichita ERS                     19
Wisconsin RS                    19
Wyoming Public Employees         7
Name: fy, Length: 146, dtype: int64

In [20]:
#Top 20 plan year counts
plan_90plus_counts.sort_values(ascending=False).head(20)

PlanName
Washington LEOFF Plan 2                    20
South Dakota RS                            20
Oklahoma City ERS                          19
Wichita ERS                                19
Washington Teachers Plan 2/3               19
NY State & Local ERS                       19
TN State and Teachers                      19
Wisconsin RS                               19
New York State Teachers                    18
NY State & Local Police & Fire             18
Louisiana Parochial Employees              18
North Carolina Local Government            17
Pennsylvania Municipal                     17
Washington PERS 2/3                        17
Delaware County and Municipal Employees    17
Milwaukee City ERS                         16
Connecticut Municipal                      16
Washington School Employees Plan 2/3       16
Los Angeles Fire and Police                16
Delaware State Employees                   15
Name: fy, dtype: int64

In [42]:
test_2=investperform_data_full[['ppd_id','PlanName','fy','EQCore_Actl',
       'EQDomesticLarge_Actl', 'EQDomesticMid_Actl', 'EQDomesticMisc_Actl',
       'EQDomesticSmall_Actl', 'EQGlobalGrowth_Actl', 'EQGlobal_Actl',
       'EQIntlActv_Actl', 'EQIntlDev_Actl', 'EQIntlEmerg_Actl']]
test_2.dropna(axis=1,how="all")

,ppd_id,PlanName,fy,EQCore_Actl,EQDomesticLarge_Actl,EQDomesticMid_Actl,EQDomesticMisc_Actl,EQDomesticSmall_Actl,EQGlobalGrowth_Actl,EQGlobal_Actl,EQIntlActv_Actl,EQIntlDev_Actl,EQIntlEmerg_Actl
0,1,Alabama ERS,2001,None,None,None,0.39930,None,None,None,None,None,None
1,1,Alabama ERS,2002,None,None,None,0.39100,None,None,None,None,None,None
2,1,Alabama ERS,2003,None,None,None,0.46080,None,None,None,None,None,None
3,1,Alabama ERS,2004,None,None,None,0.47240,None,None,None,None,None,None
4,1,Alabama ERS,2005,None,None,None,0.46130,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,184,Anchorage Police and Fire,2013,None,None,None,None,None,None,None,None,None,None
3033,184,Anchorage Police and Fire,2014,None,None,None,None,None,None,None,None,None,None
3034,184,Anchorage Police and Fire,2015,None,None,None,None,None,None,None,None,None,None
3035,184,Anchorage Police and Fire,2016,None,None,None,None,None,None,None,None,None,None


In [52]:
investperform_data_full.dropna(axis=1,how="all")

,AbsRtrn_Actl,AltInflation_Actl,AltMisc_Actl,Cash_Actl,Commod_Actl,CoveredCall_Actl,CreditOpp_Actl,DistrssedDebt_Actl,EEGroupID,EQCore_Actl,...,RealAssets_Actl,RECore_Actl,REIT_Actl,RelativeRtrn_Actl,REMisc_Actl,RENonCore_Actl,RiskParity_Actl,TierID,Timber_Actl,Total Sum
0,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
1,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
2,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
3,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
4,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3032,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
3033,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
3034,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0
3035,None,None,None,None,None,None,None,None,0,None,...,None,None,None,None,None,None,None,0,None,0.0


In [60]:
investperform_data_full.count()

AbsRtrn_Actl              413
AltInflation_Actl          37
AltMisc_Actl              263
Cash_Actl                2018
Commod_Actl               177
CoveredCall_Actl            8
CreditOpp_Actl            107
DistrssedDebt_Actl          7
DistrssedLend_Actl          0
EEGroupID                3037
EQCore_Actl                 4
EQDomesticLarge_Actl      203
EQDomesticMid_Actl         28
EQDomesticMisc_Actl      1960
EQDomesticSmall_Actl      202
EQGlobalGrowth_Actl         4
EQGlobal_Actl             183
EQIntlActv_Actl            11
EQIntlDev_Actl            166
EQIntlEmerg_Actl          275
EQIntlMisc_Actl          1991
EQIntlPass_Actl            13
EQLarge_Actl               18
EQMicro_Actl                5
EQMisc_Actl               485
EQOpportunistic_Actl        0
EQPrivate_Actl           1893
EQSecLend_Actl             31
EQSmall_Actl               21
EQSocialResp_Actl           0
Farm_Actl                  24
FIAlt_Actl                 44
FIBelowInvestGrd_Actl      12
FICash_Act

In [59]:
pd.set_option(
    'display.max_rows', 100
)